# 1 этап. Парсинг сайта the-village-ru (ветка http://www.the-village.ru/village/city/situation/)

In [ ]:
import requests
import re
from tqdm import tqdm_notebook as tqdm
from bs4 import BeautifulSoup
import time
import pandas as pd
import csv

def download_page(pageUrl):
    try:
        user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'
        page = requests.get(pageUrl, 
                             headers={'User-Agent':user_agent}) 
        html = page.content
        contents = html.decode('utf-8')
        soup = BeautifulSoup(contents, 'html.parser')
        
        title = soup.find('h1', {'class': 'article-title'})
        rubric = soup.find('li', {'class': 'item-meta meta-flow'})
        obj = soup.find('li', {'class': 'item-meta meta-posted'})
        views = soup.find('li', {'class': 'item-meta meta-views-counter'})
        comments = soup.find('li', {'class': 'item-meta meta-comments-counter'})
        content = soup.find('div', {'class': 'article-wrap'})
        
        return title, rubric, obj, views, comments, content
    except requests.HTTPError:
        return 'HTTPError at %s'%pageUrl
    except UnicodeEncodeError:
        pass
    except ConnectionError:
        time.sleep(30)
        pass

titles = []
rubrics = []
objects = []
viewsCount = []
commentsCount = []
contents = []
ids = []
commonUrl = 'http://www.the-village.ru/village/city/situation/'
for i in tqdm(range(250000, 297134)):
    pageUrl = commonUrl + str(i)
    if download_page(pageUrl) == (None, None, None, None, None, None):
        continue
    else:
        titles.append('%s'%download_page(pageUrl)[0])
        rubrics.append('%s'%download_page(pageUrl)[1])
        objects.append('%s'%download_page(pageUrl)[2])
        viewsCount.append('%s'%download_page(pageUrl)[3])
        commentsCount.append('%s'%download_page(pageUrl)[4])
        contents.append('%s'%download_page(pageUrl)[5])
        ids.append('%d'%i)
cleanTitles = []
cleanRubrics = []
cleanObjects = []
cleanestObjects =[]
cleanViewsCount = []
cleanCommentsCount = []
cleanContents = []
cleanestContents = []
for item in titles:
    data = re.sub(r'\<[^>]*\>', '', item)
    cleanTitles.append(data)
for item in rubrics:
    data = re.sub(r'\<[^>]*\>', '', item)
    cleanRubrics.append(data)
for item in objects:
    data = re.sub(r'\<[^>]*\>', ' ', item)
    cleanObjects.append(data)
for item in cleanObjects:
    data = re.sub(u'\\xa0', ' ', item)
    cleanestObjects.append(data)
for item in viewsCount:
    data = re.sub(r'\<[^>]*\>', ' ', item)
    cleanViewsCount.append(data)
for item in commentsCount:
    data = re.sub(r'\<[^>]*\>', ' ', item)
    cleanCommentsCount.append(data)
for item in contents:
    data = re.sub(r'\<[^>]*\>', ' ', item)
    cleanContents.append(data)
for item in cleanContents:
    data = re.sub(u'\\xa0', ' ', item)
    cleanestContents.append(data)
authors = []
dates = []
splObjects = []
for el in cleanestObjects:
    splObj = el.split(',')
    splObjects.append(splObj)
for el in splObjects:
    try:
        authors.append(el[0])
    except IndexError:
        authors.append('???')
    try:
        dates.append(el[1])
    except IndexError:
        dates.append('???')
raw_data = {
    'ID': ids,
    'title': cleanTitles,
    'rubric': cleanRubrics,
    'author': authors,
    'date': dates,
    'content': cleanestContents,
    'views': cleanViewsCount,
    'comments': cleanCommentsCount,
    }
df = pd.DataFrame(raw_data, columns = ['ID', 'title', 'rubric', 'author', 'date', 'content', 'views', 'comments'])
df.to_csv('village_city_situation.csv', encoding='utf-8')

In [3]:
#первоначальный датафрейм был немного почищен
df = pd.read_csv('village_city_new.csv', index_col=0)
df

,ID,title,rubric,author,date,content,views,comments
0,287600,Киноклуб Downtown Project покинет помещение на...,Новости,The Village-Владивосток,16 октября 2017 в 10:09,"\r\n \r\r\n Киноклуб Downtown Project, н...",556,0
1,287602,Автоматически взимать пошлину с интернет-покуп...,Предложение дня,Владимир Квасников,16 октября 2017 в 09:45,\r\n \r\r\n «Почта России» предложила пр...,2527,1
2,287604,Прокуратура проверяет ЦПКиО из-за запрета на ф...,Новости,Оля Татарникова,16 октября 2017 в 09:52,\r\n \r\r\n Представителей администрации...,182,0
3,287606,Telegram оштрафовали на 800 тысяч рублей за от...,Новости,Владимир Квасников,16 октября 2017 в 10:01,\r\n \r\r\n Суд оштрафовал мессенджер Te...,1871,0
4,287608,Интерьер неделиКак оборудовать офис в квартире...,None,None,???,\r\n \r\n \r\n \r\r\n Большинств...,13918,7
5,287610,«Золотой Вавилон» эвакуировали из-за угрозы вз...,Новости,Владимир Квасников,16 октября 2017 в 10:23,\r\n \r\r\n Посетителей и сотрудников то...,1449,0
6,287612,"События неделиГришковец, мастер-класс по живоп...",None,None,???,\r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\...,232,0
7,287614,В нижегородской Вышке расскажут об учебе и раб...,Новости,Марк Григорьев,16 октября 2017 в 10:32,\r\n \r\r\n Семинар карьерных экспертов ...,247,0
8,287616,До конца года между Петербургом и Шри-Ланкой о...,Новости,Сергей Феофанов,16 октября 2017 в 10:41,\r\n \r\r\n 14 октября власти Санкт-Пете...,589,0
9,287618,В движении электричек на Рижском направлении п...,Новости,Владимир Квасников,16 октября 2017 в 10:42,\r\n \r\r\n В движении электропоездов на...,494,0
